Robb Chun, Vincent Ying

ECEN 523 - Final Project

Jun 3, 2024

# Medical Research Assistant with Ollama and Crewai

https://medium.com/the-ai-forum/create-a-blog-writer-multi-agent-system-using-crewai-and-ollama-f47654a5e1cd

In [1]:
!pip install os getpass openai
!pip install langchain_openai crewai

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [3]:
# Set up OpenAI
import os
import getpass
import openai

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:········


In [5]:
# Setup LLM
llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434/v1")

In [6]:
# Topic Agent
topic = Agent(
    role="Topic Agent",
    goal="Generate topic sentences for the {topic} in a medical research paper",
    backstory=  
        """
        You're working on generating topic sentences for medical research about the topic: {topic} in 'https://medium.com/'.
        Collect information that helps the author write an engaging and factually correct research paper.
        Prepare topic sentences for the research paper.
        Your work is the basis for the Writer Agent to fill out the paragraphs on each topic sentence.
        """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Create topic task
outline = Task(
    description=(
        """
        1. Prioritize the latest, cutting edge research on {topic} that has wide interest.
        2. Keep in mind this is for publication in an academic journal.
        3. Develop an outline including introduction, key points, and conclusion.
        4. Include relevant medical books and journal articles.
        """
    ),
    expected_output="An outline of the topic sentences for each paragraph in the paper.",
    agent=topic,
)

In [8]:
# Writer Agent
writer = Agent(
    role="Paragraph Writer",
    goal="Write insightful and factually accurate paragraphs about the topic: {topic}",
    backstory=
        """
        You're writing a new research paper about {topic} in 'https://medium.com/'.
        Base your writing on the work of the Topic Agent, who provides an outline and relevant context about the overall research topic.
        Follow the main objectives and direction of the outline, as provide by the Topic Agent.
        Also provide objective and impartial insights and back them up with information provide by the Topic Agent.
        """,
    allow_delegation=False,
    llm=llm,
    verbose=True
)

# Create Writer Task
write = Task(
    description=(
        """
        1. Use the topic outline to fill out a research paper on {topic}.
        2. Sections/Subtitles are properly named and formatted.
        3. Ensure the paper is structured with an relevant introduction, insightful body, and a summarizing conclusion.
        4. Incorporate latest medical research data when possible.
        5. Proofread for grammatical errors and has the proper academic tone for journal publication.
        """
    ),
    expected_output="A well-written research paper in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [11]:
# Editor Agent
editor = Agent(
    role="Editor",
    goal="Edit a given research paper so it is ready for academic publication.",
    backstory=
        """
        You are an editor who receives a draft of a research paper from the Paragraph Writer.
        Your goal is to review the paper to ensure it can be published in a medical journal.
        """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Create Editor Task
edit = Task(
    description=("Proofread the given research paper for grammatical errors and has the proper academic tone for journal publication."),
    expected_output="A well-written research paper in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [12]:
# Creating the Crew
crew = Crew(
    agents=[topic, writer, editor],
    tasks=[outline, write, edit],
    verbose=2
)

In [13]:
# Run the crew
inputs = {"topic":"Efficacy of mRNA vaccines in leading cancers."}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Topic Agent
 [INFO]: == Starting Task: 
        1. Prioritize the latest, cutting edge research on Efficacy of mRNA vaccines in leading cancers. that has wide interest.
        2. Keep in mind this is for publication in an academic journal.
        3. Develop an outline including introduction, key points, and conclusion.
        4. Include relevant medical books and journal articles.
        


> Entering new CrewAgentExecutor chain...
Thought: I am ready to generate topic sentences for the research paper on Efficacy of mRNA vaccines in leading cancers.

Action: I will prioritize cutting-edge research, considering academic publication criteria and incorporating relevant medical books and journal articles.

Final Answer:

**I. Introduction**

1. mRNA vaccines: a revolutionary approach to cancer treatment.
2. The promise of mRNA vaccines in combating leading cancers.
3. Rationale for investigating the efficacy of mRNA vaccines in treating leading cancers.

**I

Thought: I am now ready to give a great answer. It's essential for me to edit this research paper carefully to ensure it meets the standards of a medical journal.

Final Answer:

**I. Introduction**
mRNA vaccines have revolutionized cancer treatment, offering the potential to induce targeted and durable immune responses. By selectively targeting specific cancer antigens, these vaccines can stimulate the immune system to mount a response against tumor cells, thereby improving treatment outcomes. The numerous clinical trials currently underway aim to investigate the efficacy of mRNA vaccines in treating leading cancers.

Recent advancements in mRNA vaccine technology have sparked hope that this approach can be effective against various types of cancer. The rationale behind studying the efficacy of mRNA vaccines in combating leading cancers stems from the need to explore innovative therapeutic strategies that can improve treatment outcomes, enhance patient survival, and expand our underst

In [14]:
# Display the result
from IPython.display import Markdown,display
display(Markdown(result))

**I. Introduction**
mRNA vaccines have revolutionized cancer treatment, offering the potential to induce targeted and durable immune responses. By selectively targeting specific cancer antigens, these vaccines can stimulate the immune system to mount a response against tumor cells, thereby improving treatment outcomes. The numerous clinical trials currently underway aim to investigate the efficacy of mRNA vaccines in treating leading cancers.

Recent advancements in mRNA vaccine technology have sparked hope that this approach can be effective against various types of cancer. The rationale behind studying the efficacy of mRNA vaccines in combating leading cancers stems from the need to explore innovative therapeutic strategies that can improve treatment outcomes, enhance patient survival, and expand our understanding of cancer biology.

**II. Background**
The immunology of mRNA vaccines relies on the delivery of genetic material into cells, which is then translated into a specific protein or antigen. In cancer therapy, mRNA vaccines aim to induce an immune response against tumor-associated antigens. Preclinical studies have demonstrated promising results, with many mRNA vaccine candidates demonstrating efficacy in reducing tumor growth and improving overall survival.

However, there are challenges and limitations associated with developing mRNA vaccines for cancer treatment, including concerns over immunogenicity, tumor heterogeneity, and the potential for immune evasion by tumor cells. A comprehensive understanding of these hurdles is essential to overcome them effectively.

**III. Clinical Efficacy**
Several clinical trials have investigated the efficacy of mRNA vaccine candidates in treating leading cancers such as lung, breast, and colon cancer. Studies have shown that mRNA vaccines can elicit robust immune responses against specific tumor antigens, resulting in significant reductions in tumor growth and improved overall survival rates.

Design features of mRNA vaccines, including antigen selection and adjuvant choice, have been shown to impact clinical outcomes. Additionally, patient-specific factors, such as age, comorbidity status, and disease stage, may influence mRNA vaccine response and treatment outcomes.

**IV. Mechanisms of Action**
The molecular mechanisms by which mRNA vaccines target cancer cells involve the delivery of tumor-specific antigens, which are then processed and presented to immune effector cells. This leads to the activation of antigen-specific T-cells that can recognize and eliminate cancer cells.

Modulation of immune checkpoints has been shown to enhance mRNA vaccine efficacy against leading cancers. The interplay between mRNA vaccine-induced immunity and tumor microenvironment dynamics is complex and may influence treatment outcomes, highlighting the need for further research in this area.

**V. Conclusion**
The evidence suggests that mRNA vaccines have significant potential as a promising therapeutic modality for treating leading cancers. While there are challenges associated with developing mRNA vaccines for cancer treatment, ongoing research efforts aim to overcome these limitations and improve the efficacy of mRNA vaccine development.

Identifying knowledge gaps and areas for future research will be crucial in improving the efficacy of mRNA vaccine development. By continuing to explore innovative approaches like mRNA vaccines, we may unlock new therapeutic strategies that can improve patient outcomes and expand our understanding of cancer biology.

I hope this revised version meets your expectations!